In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import time
import pandas as pd

In [3]:
# setups up chromedriver and the browser control
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# URL of page to be scraped
url = "http://www.eregulations.com/newjersey/hunting/2018-19-deer-harvest-summaries/"

# sets browser to visit the url and then wait to ensure all of the page loads
browser.visit(url)
time.sleep(5)

# builds soup object for data scraping
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [23]:
# pulls in table from the soup object
table = soup.find_all("table", class_="New-Tables")
# converts the table into pandas dataframe
hunt_df = pd.read_html(str(table))[0]
# drops row populated by table title
hunt_df = hunt_df.drop([0])
# moves row with table headers into column headers
hunt_df.columns = hunt_df.iloc[0]
hunt_df = hunt_df.drop([1])
# drop totals row
hunt_df = hunt_df.drop([62])
# drop unneeded harvest breakdown by permit type
del hunt_df["Fall Bow"]
del hunt_df["Permit Bow"]
del hunt_df["Six-day Firearm"]
del hunt_df["Permit Muzzleloader"]
del hunt_df["Permit Shotgun"]
del hunt_df["Winter Bow"]
del hunt_df["Youth Days"]
# convert DMZ into int64
hunt_df["DMZ"] = hunt_df["DMZ"].astype("int64")

In [24]:
# assign each DMZ to a county
# read in ref csv
dmz_county = "../CDC_input/dmz_county.csv"
# convert to pandas dataframe
ref_df = pd.read_csv(dmz_county)
# converts DMZ column into int64 for later merge
ref_df["DMZ"] = ref_df["DMZ"].astype("int64")
# merge county data into primary dataframe
test_df = hunt_df.merge(ref_df, on="DMZ")
test_df

,DMZ,Total Harvest,county
0,1,443,Sussex County
1,2,1596,Sussex County
2,3,360,Passaic County
3,4,393,Sussex County
4,5,2444,Warren County
5,6,793,Morris County
6,7,1731,Warren County
7,8,3088,Hunterdon County
8,9,583,Somerset County
9,10,2272,Hunterdon County


In [22]:
# rename columns

# set DMZ as index
hunt_df.dtypes

1
DMZ              object
Total Harvest    object
dtype: object